In [ ]:
using Revise

using Statistics
using Symbolics
using ArrayOperations

import Base: getindex,
             @propagate_inbounds

In [ ]:
const δ⁻ = Backward{1}(-), Backward{2}(-)
const δ⁺ = Forward{1}(-), Forward{2}(-)

const σ⁻ = Backward{1}(middle), Backward{2}(middle)
const σ⁺ = Forward{1}(middle), Forward{2}(middle);

In [ ]:
# \invw
function grad(ω, γ, a, b, ʍ)
    (δ⁻[1](ω) * b[1] + (σ⁺[1](a[1]) - b[1]) * γ - σ⁻[1](δ⁺[1](a[1]) * γ)) * ʍ[1],
    (δ⁻[2](ω) * b[2] + (σ⁺[2](a[2]) - b[2]) * γ - σ⁻[2](δ⁺[2](a[2]) * γ)) * ʍ[2]
end

function div(ω, γ, a, b)
    δ⁺[1](ω[1]) * a[1] + (σ⁻[1](b[1]) - a[1]) * γ[1] - σ⁺[1](δ⁻[1](b[1]) * γ[1]) +
    δ⁺[2](ω[2]) * a[2] + (σ⁻[2](b[2]) - a[2]) * γ[2] - σ⁺[2](δ⁻[2](b[2]) * γ[2])
end

In [ ]:
dims = 1, 1

@variables ω[dims...] γ[dims...]
@variables ax[dims...] ay[dims...]; a = (ax, ay)
@variables bx[dims...] by[dims...]; b = (bx, by)
@variables ʍx[dims...] ʍy[dims...]; ʍ = (ʍx, ʍy)

In [ ]:
qω = grad(ω, γ, a, b, ʍ)
qγ = qω

lap = div(qω, qγ, a, b)

In [ ]:
@syms i::Int j::Int

qωx, qωy = qω

string(qωx[i, j])

In [ ]:
Lij = lap[i, j]

string(Lij)

In [ ]:
Symbolics.get_variables(Lij)

In [ ]:
∂ωij = Differential(ω[i, j])

In [ ]:
∂Lij∂ωij = simplify(expand_derivatives(∂ωij(Lij)))

string(∂Lij∂ωij)

In [ ]:
struct BinOp <: Ary{2} end
      
@inline @propagate_inbounds function getindex(this::Ret{BinOp}, i::Int)
    op = operator(this)
    x, y = arguments(this)

    x[i] * y[i-1]
end

In [ ]:
f = BinOp()

In [ ]:
n = 32; x = rand(32); y = rand(32);

In [ ]:
f₀ = f(x, y)

In [ ]:
f₀ + f₀